In [8]:
!pip install pymorphy3
!pip uninstall docx
!pip install python-docx

Found existing installation: docx 0.2.4
Uninstalling docx-0.2.4:
  Would remove:
    /usr/local/docx-template/_rels/.rels
    /usr/local/docx-template/docProps/thumbnail.jpeg
    /usr/local/docx-template/word/fontTable.xml
    /usr/local/docx-template/word/numbering.xml
    /usr/local/docx-template/word/settings.xml
    /usr/local/docx-template/word/styles.xml
    /usr/local/docx-template/word/theme/theme1.xml
    /usr/local/lib/python3.11/dist-packages/docx-0.2.4.dist-info/*
    /usr/local/lib/python3.11/dist-packages/docx.py
Proceed (Y/n)? y
Y
  Successfully uninstalled docx-0.2.4


In [20]:
import re
import pymorphy3
from docx import Document

def check_keywords(text):
    morph = pymorphy3.MorphAnalyzer()
    words = text.split()

    state_registration_base = [morph.parse('государственная')[0].normal_form,
                               morph.parse('регистрация')[0].normal_form]
    rospatent_base = morph.parse('роспатент')[0].normal_form
    working_days_base = [morph.parse('рабочие')[0].normal_form,
                         morph.parse('дни')[0].normal_form]

    state_registration_found = False
    rospatent_found = False
    working_days_found = False

    registration_positions = []
    working_days_positions = []

    for index in range(len(words) - 1):
        first_word = morph.parse(words[index])[0].normal_form
        second_word = morph.parse(words[index + 1])[0].normal_form

        if first_word == state_registration_base[0] and second_word == state_registration_base[1]:
            state_registration_found = True
            registration_positions.append(index)
        if rospatent_base in morph.parse(words[index])[0].normal_form:
            rospatent_found = True
        if first_word == working_days_base[0] and second_word == working_days_base[1]:
            working_days_found = True
            working_days_positions.append(index)

    results = {
        "государственная регистрация": state_registration_found,
        "роспатент": rospatent_found,
        "рабочие дни": working_days_found,
    }

    for pos in registration_positions:
        start = max(0, pos - 40)
        end = min(len(words), pos + 2 + 40)
        substring = ' '.join(words[start:end])
        pattern = r'(d+)s+рабочихs+дней'
        matches = re.findall(pattern, substring)



    for pos in working_days_positions:
        start = max(0, pos - 40)
        end = min(len(words), pos + 2 + 40)
        substring = ' '.join(words[start:end])
        pattern = r'(d+)s+рабочиеs+дни'
        matches = re.findall(pattern, substring)

        if matches:
            results["рабочие дни"].extend(matches)

    return results

if __name__ == "__main__":

    source = Document("РЕГИСТРАЦИЯ.docx")
    document_text = ''
    paras = source.paragraphs
    for para in paras:
        document_text += f'{para.text}\n'

    results = check_keywords(document_text)

    for keyword, found in results.items():
        if isinstance(found, list) and found:
            print(f'Найдены следующие значения для "{keyword}": {", ".join(found)}')
        elif found:
            print(f'Слово "{keyword}" найдено в документе.')
        else:
            print(f'Слово "{keyword}" не найдено в документе.')




Слово "государственная регистрация" найдено в документе.
Слово "роспатент" найдено в документе.
Слово "рабочие дни" найдено в документе.
